## 5.1 template vs macro

In [1]:
#define MAX(x, y) (((x) > (y)) ? (x) : (y))

In [2]:
int a = 0;
int b = 1;
MAX(a++, b++)

(int) 2


In [3]:
#include <concepts>
#include <array>

In [4]:
template <typename T>
concept Comparable = requires(T a, T b){
    { a > b } -> std::same_as<bool>;
};

In [5]:
struct Foo{};

In [6]:
template<typename T>
T max(T a, T b) {return a > b ? a : b ;}

see compiler complaints below:

In [7]:
// max(Foo{}, Foo{});

see compiler complaints below:

In [8]:
template<typename T>
requires Comparable<T>
T comparable_max(T a, T b) {return a > b ? a : b ;}

In [9]:
// comparable_max(Foo{}, Foo{});

## 5.2 template class meta function

In [10]:
using Array5x4x3 = std::array<std::array<std::array<int, 3>, 4>, 5>;

In [11]:
template<typename T, size_t I, size_t ...Is>
struct Array{
    using type = std::array<typename Array<T, Is...>::type, I>;
};

// terminating condition:
template<typename T, size_t I>
struct Array<T, I>{
    using type = std::array<T, I>;
};


In [12]:
static_assert(std::is_same_v<Array<int, 5, 4, 3>::type, Array5x4x3>);

## 5.3 TypeList

In [13]:
template<typename ...Ts>
struct TypeList{};

using List = TypeList<int, double>;

In [14]:
using One = std::integral_constant<int, 1>;
constexpr auto one = One::value;
using Two = std::integral_constant<int, 2>;
constexpr auto two = Two::value;

In [15]:
using IntegralList = TypeList<One, Two>;

In [16]:
template <typename ...Ts>
struct TypeListBase {
    struct IsTypeList{};
    using type = TypeListBase;
    constexpr static size_t size = sizeof...(Ts);
    template<typename ...T> using append = TypeListBase<Ts..., T...>;
    template<typename ...T> using prepend = TypeListBase<T..., Ts...>;
    template<template<typename ...> typename T> using to = T<Ts...>;
};

template<typename TypeListBase>
concept TL = requires{
    typename TypeListBase::IsTypeList;
    typename TypeListBase::type;
};

In [17]:
using AList = TypeListBase<int, char>;

In [18]:
static_assert(TL<AList>);
static_assert(AList::size == 2);
static_assert(std::is_same_v<AList::prepend<double>, TypeListBase<double, int, char>>);
static_assert(std::is_same_v<AList::append<double>, TypeListBase<int, char, double>>);

more advanced functions:

map:

In [19]:
template<TL In, template<typename> class F>
struct Map;
template<template<typename> class F, typename ...Ts>
struct Map<TypeListBase<Ts...>, F> : TypeListBase<typename F<Ts>::type...>{};

In [20]:
template<TL In, template<typename> class F>
using Map_t = typename Map<In, F>::type;

In [21]:
using LongList = TypeListBase<char, float, double, int, char>;

In [22]:
static_assert(std::is_same_v<Map_t<LongList, std::add_pointer>, TypeListBase<char*, float*, double*, int*, char*>>)

filter:

In [23]:
template<TL In, template <typename> class P, TL Out = TypeListBase<>>
struct Filter : Out {};
template<template <typename> class P, TL Out, typename H, typename ...Ts>
struct Filter <TypeListBase<H, Ts...>, P, Out> : 
    std::conditional_t< P<H>::value, 
        Filter<TypeListBase<Ts...>, P, typename Out::template append<H>>, 
        Filter<TypeListBase<Ts...>, P, Out>>{};

In [24]:
template<TL In, template <typename> class P, TL Out = TypeListBase<>>
using Filter_t = Filter<In, P>::type;

In [25]:
template<typename T>
using SizeLessThan4 = std::bool_constant< (sizeof(T) < 4)>;

In [26]:
static_assert( std::is_same_v< Filter_t<LongList, SizeLessThan4>, TypeListBase<char, char> >);

fold: 

In [27]:
template<typename T>
struct Return {using type = T;};

In [28]:
template<TL In, typename Init, template<typename, typename> class Op>
struct Fold : Return <Init>{};

template<typename Acc, template<typename, typename> class Op, typename H, typename ...Ts>
struct Fold<TypeListBase<H, Ts...>, Acc, Op> : Fold<TypeListBase<Ts...>, typename Op<Acc, H>::type, Op> {};

In [29]:
template<TL In, typename Init, template<typename, typename> class Op>
using Fold_t = Fold<In, Init, Op>::type;

In [30]:
template<typename Acc, typename E>
using TypeSizeAcc = std::integral_constant<size_t, Acc::value + sizeof(E)>;

In [31]:
static_assert(Fold_t<LongList, std::integral_constant<size_t, 0>, TypeSizeAcc>::value == 18);

concat:

In [32]:
template<TL...In> struct Concat;
template<TL...In> using Concat_t = typename Concat<In...>::type;

In [33]:
template<> struct Concat<> : TypeListBase<> {};

In [34]:
template<TL In> struct Concat<In> : In {};

In [35]:
template<TL In1, TL In2> struct Concat<In1, In2> : In2::template to<In1::template append> {};

In [36]:
template<TL In1, TL In2, TL...Ins>
struct Concat<In1, In2, Ins...> : Concat<Concat<In1, In2>, Ins...> {};

In [37]:
static_assert(std::is_same_v< Concat_t< TypeListBase<int, double>, TypeListBase<char, float>>, 
    TypeListBase<int, double, char, float>>);

elem:

In [38]:
template<TL In, typename E>
class Elem{
    template<typename Acc, typename T>
    using FindE = std::conditional_t<Acc::value, Acc, std::is_same<T, E>>;
    using Found = Fold_t<In, std::false_type, FindE>;
public:
    constexpr static bool value = Found::value;
};

In [39]:
template<TL In, typename E>
struct Element : std::false_type {};

template<typename E, typename ...Ts>
struct Element<TypeListBase<Ts...>, E>:
    std::bool_constant< (false || ... || std::is_same_v<E, Ts>) > {};

In [40]:
static_assert(Elem<LongList, char>::value);

In [41]:
static_assert(not Element<LongList, long long>::value);

unique:

In [42]:
template<TL In>
class Unique{
    template<TL Acc, typename E>
    using Append = std::conditional_t<Elem<Acc, E>::value, Acc, typename Acc::template append<E>>;
public:
    using type = Fold_t<In, TypeListBase<>, Append>;
};

In [43]:
template<TL In>
using Unique_t = Unique<In>::type;

In [44]:
static_assert(std::is_same_v<Unique_t<LongList>, TypeListBase<char, float, double, int>>);

partition:

In [45]:
template<TL In, template<typename> typename P>
class Partition{
    template<typename Arg>
    using NotP = std::bool_constant<not P<Arg>::value>;
public:
    struct type{
        using Satisfied = Filter_t<In, P>;
        using Rest = Filter_t<In, NotP>;
    };
};

In [46]:
using SplitBySize4 = Partition<LongList, SizeLessThan4>;

In [47]:
static_assert(std::is_same_v<SplitBySize4::type::Satisfied, TypeListBase<char, char>>);
static_assert(std::is_same_v<SplitBySize4::type::Rest, TypeListBase<float, double, int>>);

sort:

In [48]:
template<TL In, template<typename, typename> typename Cmp>
struct Sort : TypeListBase<>{};

template<template<typename, typename> typename Cmp, typename Pivot, typename ...Ts>
class Sort<TypeListBase<Pivot, Ts...>, Cmp>{
    template<typename E>
    using LT = Cmp<E, Pivot>;
    using P = Partition<TypeListBase<Ts...>, LT>::type;
    using SmallerSorted = typename Sort<typename P::Satisfied, Cmp>::type;
    using BiggerSorted = typename Sort<typename P::Rest, Cmp>::type;
public:
    using type = Concat_t<typename SmallerSorted::template append<Pivot>, BiggerSorted>;
};

In [49]:
template<typename L, typename R>
using SizeCmp = std::bool_constant<sizeof(L) < sizeof(R)>;

In [50]:
static_assert(std::is_same_v<Sort<LongList, SizeCmp>::type, TypeListBase<char, char, float, int, double>>);

## 5.4 exercise